In [18]:
import torch
from utils import get_embedding
import pandas as pd
from tqdm import tqdm
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import csv

In [13]:
model_name = "text2vec-base"

In [14]:
embedding = get_embedding(model_name=model_name)

In [15]:
t = "刷美德：英国80后为何越来越传统"

In [19]:
fn = "thucnews/thucnews.csv"
df = pd.read_csv(fn)
df.shape

(3163, 11)

In [20]:
title_embeddings = torch.zeros(df.shape[0], 768)
title_embeddings.shape

torch.Size([3163, 768])

In [23]:
titles = df['title']
with torch.no_grad():
    i = 0
    for title in tqdm(titles):
        t_em = torch.tensor(embedding.embed_query(title), dtype=float)
        title_embeddings[i, :] = t_em
        i = i + 1
    print(i)

100%|██████████| 3163/3163 [01:16<00:00, 41.51it/s]

3163


In [3]:
with open("thucnews/views/pkl/title_embeddings.pickle", "rb") as file:
    title_embeddings = pickle.load(file)

In [4]:
title_embeddings.shape

torch.Size([3163, 768])

In [5]:
score = cosine_similarity(title_embeddings, title_embeddings)

In [14]:
percentage = np.sum(score > 0.26 ) / (score.size)
percentage

0.7443747951560932

In [12]:
score

array([[1.0000001 , 0.1604101 , 0.30740777, ..., 0.32357004, 0.19480887,
        0.267757  ],
       [0.1604101 , 1.0000001 , 0.11000511, ..., 0.17619243, 0.29722834,
        0.20300844],
       [0.30740777, 0.11000511, 1.0000002 , ..., 0.2870033 , 0.33224702,
        0.3169097 ],
       ...,
       [0.32357007, 0.17619246, 0.28700334, ..., 0.9999999 , 0.27789873,
        0.33564255],
       [0.19480883, 0.29722837, 0.33224705, ..., 0.27789873, 1.        ,
        0.25049067],
       [0.26775697, 0.20300843, 0.31690958, ..., 0.33564252, 0.25049067,
        1.0000001 ]], dtype=float32)

In [15]:
score[score<=0.26] = 0

In [16]:
np.fill_diagonal(score, 1.0)

In [17]:
save_path = "thucnews/views/title_views.csv"
# 打开CSV文件
with open(save_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # 写入表头
    # writer.writerow(['Row', 'Column', 'Value'])

    # 遍历数组并写入CSV文件
    for i in tqdm(range(score.shape[0])):
        for j in range(score.shape[1]):
            if score[i][j] > 0:
                writer.writerow([i, j, score[i, j]])